In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Deliverable 1 — A Jupyter notebook with a description of the problem/data, exploratory data analysis (EDA) procedure, analysis (model building and training), result, and discussion/conclusion. 

Suppose your work becomes so large that it doesn’t fit into one notebook (or you think it will be less readable by having one large notebook). In that case, you can make several notebooks or scripts in a GitHub repository (as deliverable 3) and submit a report-style notebook or pdf instead. 

If your project doesn’t fit into Jupyter notebook format (E.g., you built an app that uses ML), write your approach as a report and submit it in a pdf form. 

Deliverable 2 — A public project GitHub repository with your work (please also include the GitHub repo URL in your notebook/report).

Deliverable 3 — A screenshot of your position on the Kaggle competition leaderboard for your top-performing model.

Step 1.
Brief Description of the problem and ddata
Brief description of the problem and data (5 pts) 

Briefly describe the challenge problem and NLP. Describe the size, dimension, structure, etc., of the data. 

Step 2.
Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data (15 pts)

Show a few visualizations like histograms. Describe any data cleaning procedures. Based on your EDA, what is your plan of analysis? 

Step 3.
Model Architecture
Describe your model architecture and reasoning for why you believe that specific architecture would be suitable for this problem. Compare multiple architectures and tune hyperparameters. 

Step 4.
Results and Analysis (35 pts) 

Run hyperparameter tuning, try different architectures for comparison, apply techniques to improve training or performance, and discuss what helped.

Includes results with tables and figures. There is an analysis of why or why not something worked well, troubleshooting, and a hyperparameter optimization procedure summary.

Step 5. 
Conclusion (15 pts)

Discuss and interpret results as well as learnings and takeaways. What did and did not help improve the performance of your models? What improvements could you try in the future?

Step 6.
Produce Deliverables: High-Quality, Organized Jupyter Notebook Report, GitHub Repository, and screenshot of Kaggle leaderboard (35 points)

These deliverables serve two purposes- grade for this course and your project portfolio that you can show when you apply for jobs.

If you haven’t used GitHub previously, please find a tutorial and get acquainted with it before the project deadline. For the sake of this project, you can use GitHub to showcase your codebase. In the real world, versioning with GitHub is vital for collaboration. Sometimes Jupyter notebooks don’t seem particularly well-suited to versioning with GitHub due to hard-to-read diffs and the like. If you want to use this project as an opportunity to practice versioning with GitHub, consider something like the following: 

**Part 1: Brief Description of the Problem and Data**
This project uses the kaggle competition dataset https://www.kaggle.com/competitions/histopathologic-cancer-detection/overview. The goal is to use convolutional neural networks (CNN) to identify metastatic cancer in small image patches. The data are RBG images, and the image files are .tif files. The images are of cells that will be labeled cancer or not having cancer, and our job is to perform image classification to predict the labels of the test images.

**Part 2. Exploratory Data Analysis (EDA) — Inspect, Visualize and Clean the Data**

Load the libraries


In [ ]:
import matplotlib.pyplot as plt

import time

import cv2 # For EDA

from PIL import Image

import sys

from numpy.random import seed
seed(101)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam


from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
tf.random.set_seed(101)

Load Data

In [ ]:
train_labels_df = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
train_labels_df.shape

Check for duplicate entries

In [ ]:
train_labels_df[train_labels_df.duplicated(keep=False)]

No duplicated were found

We will take a look at the distribution of data (cancer vs non-cancerous)

In [ ]:
train_labels_df['label'].value_counts()

The dataset favors non-cancer (0), vs cancer (1) and not too skewed in terms of distribution
Let's take a look at the images and see if there is anything that pops out at us. I highly doubt a visual inspection will help us, as cancer detection is extremely hard to an untrained eye.

In [ ]:
fig = plt.figure(figsize=(25, 8))
train_imgs = os.listdir("/kaggle/input/histopathologic-cancer-detection/train")
for idx, img in enumerate(np.random.choice(train_imgs, 40)):
    ax = fig.add_subplot(4, 10, idx+1, xticks=[], yticks=[])
    im = Image.open("/kaggle/input/histopathologic-cancer-detection/train/" + img)
    plt.imshow(im)
    lab = train_labels_df.loc[train_labels_df['id'] == img.split('.')[0], 'label'].values[0]
    ax.set_title(f'Label: {lab}')

As expected, there is nothing that is obvious for identifying the cancer.....
Based on our EDA, we found no duplicates an extremely large number of images (220025 for training), and relatively even distribution of classification.
For the analysis and model creation, I plan to lower the sample size drastically for performance purposes so we can do our analysis faster.

In [ ]:
IMAGE_SIZE=96
IMAGE_CHANNELS=3
SAMPLE_SIZE=2000  

In [ ]:
'''df0=train_labels_df[train_labels_df['label']==0].sample(SAMPLE_SIZE,random_state=101)
df1=train_labels_df[train_labels_df['label']==1].sample(SAMPLE_SIZE,random_state=101)
df_data = pd.concat([df0, df1], axis=0) #.reset_index(drop=True)
df_data = shuffle(df_data)
y = df_data['label']
df_train, df_val = train_test_split(df_data, test_size=0.20, random_state=101, stratify=y)
print(df_train.shape)
print(df_val.shape)'''

In [ ]:
'''base_dir='base_dir'
os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)
no_cancer = os.path.join(train_dir, 'no_cancer')
os.mkdir(no_cancer)
yes_cancer = os.path.join(train_dir, 'yes_cancer')
os.mkdir(yes_cancer)
no_cancer = os.path.join(val_dir, 'no_cancer')
os.mkdir(no_cancer)
yes_cancer = os.path.join(val_dir, 'yes_cancer')
os.mkdir(yes_cancer)

'''

In [ ]:
'''os.listdir('base_dir/train_dir')'''

In [ ]:
'''df_data.head()'''

In [ ]:
'''df_data.set_index('id', inplace=True)
train_list = list(df_train['id'])
val_list = list(df_val['id'])
for image in train_list:
    
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image + '.tif'
    # get the label for a certain image
    target = df_data.loc[image,'label']
    
    # these must match the folder names
    if target == 0:
        label = 'no_cancer'
    if target == 1:
        label = 'yes_cancer'
    
    # source path to image
    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/train', fname)
    # destination path to image
    dst = os.path.join(train_dir, label, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)


# Transfer the val images

for image in val_list:
    
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image + '.tif'
    # get the label for a certain image
    target = df_data.loc[image,'label']
    
    # these must match the folder names
    if target == 0:
        label = 'no_cancer'
    if target == 1:
        label = 'yes_cancer'
    

    # source path to image
    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/train', fname)
    # destination path to image
    dst = os.path.join(val_dir, label, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)
'''

Set up the Generators

In [ ]:
'''train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'
test_path = '/kaggle/input/histopathologic-cancer-detection/test'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)
'''

In [ ]:
'''kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))

model.summary()'''

In [ ]:
'''model.compile(Adam(learning_rate=0.0001), loss='binary_crossentropy', 
              metrics=['accuracy'])
print(val_gen.class_indices)'''

In [ ]:
'''filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit(train_gen, 
                    validation_data=val_gen,
                    epochs=20, verbose=1,
                   callbacks=callbacks_list)'''

In [ ]:
'''model.metrics_names'''

In [ ]:
'''val_loss, val_acc = model.evaluate(test_gen, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_acc:', val_acc)'''

In [ ]:
'''
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()'''

In [ ]:
'''test_gen.class_indices'''

In [ ]:
'''predictions = model.predict(test_gen, steps=len(df_val), verbose=1)'''


In [ ]:
'''predictions.shape'''

In [ ]:
'''df_preds = pd.DataFrame(predictions, columns=['no_cancer', 'yes_cancer'])

df_preds.head()'''

In [ ]:
'''y_true = test_gen.classes

y_pred = df_preds['yes_cancer']
from sklearn.metrics import roc_auc_score

roc_auc_score(y_true, y_pred)'''

It looks like we have a good model for just using a small portion of the training data.
We will clear the directories and train the model with the larger dataset.
We can also tune the hyperparameters with the larger dataset and find the optimal values.

In [ ]:
'''shutil.rmtree('base_dir')'''

Below is all just setting up the training and test data with folders. Same as before.

In [ ]:
'''SAMPLE_SIZE=10000
df_data = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
df0=df_data[df_data['label']==0].sample(SAMPLE_SIZE,random_state=42)
df1=df_data[df_data['label']==1].sample(SAMPLE_SIZE,random_state=42)
df_data = pd.concat([df0, df1], axis=0).reset_index(drop=True)
df_data = shuffle(df_data)
y = df_data['label']
df_train, df_val = train_test_split(df_data, test_size=0.20, random_state=42, stratify=y)
base_dir='base_dir'
os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)
no_cancer = os.path.join(train_dir, 'no_cancer')
os.mkdir(no_cancer)
yes_cancer = os.path.join(train_dir, 'yes_cancer')
os.mkdir(yes_cancer)
no_cancer = os.path.join(val_dir, 'no_cancer')
os.mkdir(no_cancer)
yes_cancer = os.path.join(val_dir, 'yes_cancer')
os.mkdir(yes_cancer)
df_data.set_index('id', inplace=True)
train_list = list(df_train['id'])
val_list = list(df_val['id'])
for image in train_list:
    fname = image + '.tif'
    target = df_data.loc[image,'label']
    if target == 0:
        label = 'no_cancer'
    if target == 1:
        label = 'yes_cancer'
    
    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/train', fname)
    dst = os.path.join(train_dir, label, fname)
    shutil.copyfile(src, dst)

for image in val_list:
    
    fname = image + '.tif'
    target = df_data.loc[image,'label']
    
    if target == 0:
        label = 'no_cancer'
    if target == 1:
        label = 'yes_cancer'

    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/train', fname)
    dst = os.path.join(val_dir, label, fname)
    shutil.copyfile(src, dst)'''

This took about 10 minutes using a Kaggle GPU.
Now, we set up the model.

In [ ]:
'''# Set up the generators
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'
test_path = '/kaggle/input/histopathologic-cancer-detection/test'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))

model.summary()
'''


HyperParameter tuning: Vary the learning rate and find the best model.

In [ ]:
'''learning_rates=[0.0001, 0.001, 0.01]

filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]


curr_best_model = (None, 0)
curr_best_history = None
for lr_iter in learning_rates:
    model.compile(Adam(learning_rate=lr_iter), loss='binary_crossentropy', 
              metrics=['accuracy'])
    this_history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=20, verbose=1,
                   callbacks=callbacks_list)
    if this_history.history["accuracy"][-1] > curr_best_model[1]:
        curr_best_model = (model, this_history.history["accuracy"][-1])
        curr_best_history = this_history.history'''

# Using the GPU on kaggle is about 20x faster than using CPU.
Let's look at the trained model

<h1>Part 4: Results and Analysis</h1>
We trained the model and tuned learning rate hyperparameter. We optimized for validation accuracy to find the optimal parameters.

A learning rate of 0.0001 was found to be optimal.
Sometimes we have to start from here and plug in the learning rate instead of referencing curr_best_model if the connection to kaggle is lost.
Skip this next step if the model was trained and connection was not lost to kaggle

In [ ]:
# Set up the generators
SAMPLE_SIZE=10000
'''df_data = pd.read_csv('/kaggle/input/histopathologic-cancer-detection/train_labels.csv')
df0=df_data[df_data['label']==0].sample(SAMPLE_SIZE,random_state=42)
df1=df_data[df_data['label']==1].sample(SAMPLE_SIZE,random_state=42)
df_data = pd.concat([df0, df1], axis=0).reset_index(drop=True)
df_data = shuffle(df_data)
y = df_data['label']
df_train, df_val = train_test_split(df_data, test_size=0.20, random_state=42, stratify=y)
base_dir='base_dir'
os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)
no_cancer = os.path.join(train_dir, 'no_cancer')
os.mkdir(no_cancer)
yes_cancer = os.path.join(train_dir, 'yes_cancer')
os.mkdir(yes_cancer)
no_cancer = os.path.join(val_dir, 'no_cancer')
os.mkdir(no_cancer)
yes_cancer = os.path.join(val_dir, 'yes_cancer')
os.mkdir(yes_cancer)
df_data.set_index('id', inplace=True)
train_list = list(df_train['id'])
val_list = list(df_val['id'])
for image in train_list:
    fname = image + '.tif'
    target = df_data.loc[image,'label']
    if target == 0:
        label = 'no_cancer'
    if target == 1:
        label = 'yes_cancer'
    
    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/train', fname)
    dst = os.path.join(train_dir, label, fname)
    shutil.copyfile(src, dst)

for image in val_list:
    
    fname = image + '.tif'
    target = df_data.loc[image,'label']
    
    if target == 0:
        label = 'no_cancer'
    if target == 1:
        label = 'yes_cancer'

    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/train', fname)
    dst = os.path.join(val_dir, label, fname)
    shutil.copyfile(src, dst)
    
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'
test_path = '/kaggle/input/histopathologic-cancer-detection/test'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))


model.compile(Adam(learning_rate=0.0001), loss='binary_crossentropy', 
              metrics=['accuracy'])
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit(train_gen, 
                    validation_data=val_gen,
                    epochs=20, verbose=1,
                   callbacks=callbacks_list)'''

Let's take a look at the results and plot the curves


In [ ]:
#import tensorflow as tf
#model.save('/kaggle/working/my_model.h5')


In [ ]:
#val_loss, val_acc = model.evaluate(test_gen, 
 #                       steps=len(df_val))

#print('val_loss:', val_loss)
#print('val_acc:', val_acc)

Display loss and accuracy curves

In [ ]:
'''acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()'''

Make predictions

In [ ]:
#predictions = model.predict(test_gen, steps=len(df_val), verbose=1)

In [ ]:
#predictions.shape

In [ ]:
#df_preds = pd.DataFrame(predictions, columns=['no_cancer', 'yes_cancer'])

#df_preds.head()

Let's take a look at the accuracy of predictions

In [ ]:
'''from sklearn.metrics import roc_auc_score
y_true = test_gen.classes
y_pred = df_preds['yes_cancer']
roc_auc_score(y_true, y_pred)'''

Set up test directory for predictions and submission

In [ ]:
#shutil.rmtree('base_dir')

Have to get nested directory for keras to work

In [ ]:
test_dir = 'test_dir'
os.mkdir(test_dir)
test_images = os.path.join(test_dir, 'test_images')
os.mkdir(test_images)
os.listdir('test_dir')

test_list = os.listdir('/kaggle/input/histopathologic-cancer-detection/test')

for image in test_list:
    
    fname = image
    
    src = os.path.join('/kaggle/input/histopathologic-cancer-detection/test', fname)
    dst = os.path.join(test_images, fname)
    shutil.copyfile(src, dst)

num_test_images=len(os.listdir('test_dir/test_images'))
len(os.listdir('test_dir/test_images'))

In [ ]:
num_test_images=len(os.listdir('/kaggle/input/histopathologic-cancer-detection/test'))
len(os.listdir('/kaggle/input/histopathologic-cancer-detection/test'))

Point to correct directory

In [ ]:
test_path ='test_dir'
datagen = ImageDataGenerator(rescale=1.0/255)
test_gen = datagen.flow_from_directory(test_path,
                                        target_size=(96,96),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

In [ ]:
from tensorflow.keras.models import load_model
import keras
model = keras.models.load_model('/kaggle/input/modelh5/keras/default/1/my_model.h5')

In [ ]:
predictions = model.predict(test_gen, steps=num_test_images, verbose=1)

In [ ]:
len(predictions)

In [ ]:
df_preds = pd.DataFrame(predictions, columns=['no_cancer', 'yes_cancer'])
df_preds.head()

Set up format for output document. Extract file name without extension or path

In [ ]:
test_filenames = test_gen.filenames
df_preds['file_names'] = test_filenames

In [ ]:
def extract_id(x):
    extract_id = x.split('/')[1].split('.')[0]
    return extract_id
test_filenames = test_gen.filenames
submission = pd.DataFrame({'id':df_preds['file_names'].apply(extract_id), 
                           'label':df_preds['yes_cancer'], 
                          }).set_index('id')
submission.head()
submission.to_csv('/kaggle/working/submission.csv')

References:
https://keras.io/examples/vision/image_classification_from_scratch/
